In [3]:
import glob
import os
import pandas as pd
import tifffile as tiff
import cv2
import numpy as np 
from cellpose import models, io
from utils import remove_zero_pad

metadata_file = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Evaluations/Metadata/metadata.csv"
base_dir = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
out_dir = base_dir + "/_Data_Analysis/_tmp_daria/Segmentation/Annotation/Cellpose/30_percent_downscaled"

meta_data = pd.read_csv(metadata_file)

meta_data = meta_data.drop(meta_data.index[meta_data['exclude'] != 'no'])
meta_data.reset_index(drop=True, inplace=True)

model = models.Cellpose(gpu=True, model_type='cyto')
channels = [0,0]


dim_large = 4068
dim_small = 1343
#dim_small = 512

for s in meta_data['Sample']:
    sample_dir = glob.glob(base_dir + '/*{}*'.format(s))[0]
    try:
        filename = glob.glob(sample_dir + '/*/*Spot*-B*')[0]
    except:
        raise Exception(f"No ROIs found in '{sample_dir}'")

    img_cut = remove_zero_pad(tiff.imread(filename))[:dim_large, :dim_large]
    
    img_small = cv2.resize(img_cut, dsize=(dim_small,dim_small)) 
    tiff.imwrite(os.path.join(out_dir, "raw_cut", s + ".tiff"), img_small)

    masks, flows, styles, diams = model.eval(img_small, diameter=30, channels=channels)
    
    #masks_large = cv2.resize(masks, dsize=(dim_large,dim_large), interpolation = cv2.INTER_NEAREST)
    #flows_large = cv2.resize(flows[0], dsize=(dim_large,dim_large))
    tiff.imwrite(os.path.join(out_dir, "masks", s + ".tiff"), masks)
    tiff.imwrite(os.path.join(out_dir, "flows", s + ".tiff"), flows[0])







KeyboardInterrupt: 